In [1]:
from __future__ import annotations
import os
import json
from typing import Any, Dict, List, Optional, TypedDict, Literal

from pydantic import BaseModel, Field

# LangChain / OpenAI / Pinecone
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pinecone import Pinecone

# Web search (optional)
import requests

# LangGraph
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver


In [2]:
DEFAULT_INDEXES: Dict[str, str] = {
    "acts": "act-agentic-chunking",
    "rules": "rules-agentic-chunking",
    "sro": "sro-agentic-chunking",
    "ordinances": "ordinance-agentic-chunking",
}

ALLOWED_DOMAINS_DEFAULT = [
    "bdlaws.minlaw.gov.bd",
    "nbr.gov.bd",
    "mof.gov.bd",
    "dpp.gov.bd",
    "cabinet.gov.bd",
    "bangladesh.gov.bd",
]


In [3]:
class EntityHierarchy(TypedDict, total=False):
    part: Optional[str]
    chapter: Optional[str]
    section: Optional[str]
    subsection: Optional[str]
    clause: Optional[str]

class Entities(TypedDict, total=False):
    law_name: Optional[str]
    law_year: Optional[str]
    hierarchy: EntityHierarchy

class Hit(TypedDict, total=False):
    index: Literal["acts", "rules", "sro", "ordinances", "web"]
    id: str
    score: float
    text: str
    metadata: Dict[str, Any]

class Plan(TypedDict, total=False):
    need_rules: bool
    need_sro: bool
    need_ordinances: bool
    need_web: bool
    targeted_queries: List[str]

class Verification(TypedDict, total=False):
    notes: str
    risk_level: Literal["low", "medium", "high"]
    must_check: List[str]

class RAGState(TypedDict, total=False):
    query: str
    use_web: bool
    entities: Entities
    acts_hits: List[Hit]
    rules_hits: List[Hit]
    sro_hits: List[Hit]
    ordinances_hits: List[Hit]
    web_hits: List[Hit]
    stage1_answer: str
    citations_stage1: List[Dict[str, Any]]
    plan: Plan
    merged_hits: List[Hit]
    verification: Verification
    final_answer: str
    citations: List[Dict[str, Any]]

class Deps(BaseModel):
    indexes: Dict[str, str] = Field(default_factory=lambda: DEFAULT_INDEXES.copy())
    embedding_model: str = "text-embedding-3-large"
    model_fast: str = "gpt-5-mini"
    model_reason: str = "gpt-5"
    allowed_domains: List[str] = Field(default_factory=lambda: ALLOWED_DOMAINS_DEFAULT.copy())
    tavily_endpoint: str = "https://api.tavily.com/search"

    # runtime clients:
    pc: Any = None
    emb: Any = None
    llm_fast: Any = None
    llm_reason: Any = None

    def init_clients(self) -> None:
        if not os.environ.get("OPENAI_API_KEY"):
            raise RuntimeError("Please set OPENAI_API_KEY")
        if not os.environ.get("PINECONE_API_KEY"):
            raise RuntimeError("Please set PINECONE_API_KEY")
        self.pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
        self.emb = OpenAIEmbeddings(model=self.embedding_model)
        self.llm_fast = ChatOpenAI(model=self.model_fast, temperature=0.2)
        self.llm_reason = ChatOpenAI(model=self.model_reason, temperature=0.2)


In [21]:
ENTITIES_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "You are a precise legal entity extractor for Bangladeshi laws. "
               "Extract structured fields from the user query. Respond in strict JSON."),
    ("human",
     "User query:\n{query}\n\n"
     "Return JSON with keys exactly: law_name, law_year, hierarchy.\n"
     "hierarchy must be an object with optional keys: part, chapter, section, subsection, clause.\n"
     "If unknown, leave them null.\n"
     "Return exactly this shape:\n"
     "{{\"law_name\": null, \"law_year\": null, \"hierarchy\": {{\"part\": null, \"chapter\": null, \"section\": null, \"subsection\": null, \"clause\": null}}}}")
])

SYNTH_STAGE1 = ChatPromptTemplate.from_messages([
    ("system",
     "You are a meticulous Bangladeshi legal assistant. Answer using ONLY the ACTS context.\n"
     "Be concise, cite exact sections if visible in context. If unclear, say what else is needed.\n"),
    ("human",
     "User question:\n{query}\n\n"
     "ACTS context (top passages):\n{context}\n\n"
     "Write a short, precise answer in Bangla if the question is Bangla; otherwise in English.\n"
     "Add a one-line 'Next step' if the answer likely depends on Rules/SRO/Ordinances.")
])

ESCALATE_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "You are a policy decider. Return STRICT JSON."),
    ("human",
     "User question:\n{query}\n\n"
     "Initial answer (acts-only):\n{answer}\n\n"
     "Decide if we should search Rules/SRO/Ordinances and possibly the Web.\n"
     "Return JSON EXACTLY in this shape:\n"
     "{{\"need_rules\": true, \"need_sro\": true, \"need_ordinances\": false, \"need_web\": true, \"targeted_queries\": []}}")
])
VERIFY_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "You are verifying if any SRO/Gazette has amended or superseded the cited sections. "
     "Respond with STRICT JSON only (no extra text)."),
    ("human",
     "User question:\n{query}\n\n"
     "Context snippets (Acts/Rules/SRO/Web):\n{context}\n\n"
     "Return JSON EXACTLY in this shape:\n"
     "{{\"notes\": \"...\", \"risk_level\": \"low|medium|high\", \"must_check\": [\"...\"]}}")
])

FINAL_SYNTH = ChatPromptTemplate.from_messages([
    ("system",
     "You are a senior Bangladeshi legal assistant. Synthesize a precise, practical answer "
     "using Acts first, then Rules/SRO/Ordinances. Be conservative where uncertainty exists."),
    ("human",
     "User question:\n{query}\n\n"
     "Merged context (top excerpts):\n{context}\n\n"
     "Verification notes:\n{verify}\n\n"
     "Write a final answer (Bangla if the question is Bangla, else English). "
     "Include short bullet citations like [Act §X], [Rule Y], [SRO Z/Year], or a source URL.")
])


In [5]:
def _strip_json(raw: str) -> str:
    return raw.strip().removeprefix("```json").removesuffix("```").strip("` \n")

def embed(deps: Deps, text: str) -> List[float]:
    return deps.emb.embed_query(text)

def pc_query(deps: Deps, index_name: str, vector: List[float], top_k: int = 10) -> List[Dict[str, Any]]:
    idx = deps.pc.Index(index_name)
    res = idx.query(vector=vector, top_k=top_k, include_metadata=True)
    return res.get("matches", [])

def build_queries(state: RAGState) -> List[str]:
    q = state["query"]
    ents = state.get("entities", {}) or {}
    law_name = ents.get("law_name")
    law_year = ents.get("law_year")
    hier = ents.get("hierarchy", {}) or {}

    seeds = [q]
    if law_name:
        seeds += [
            f"{law_name} {law_year or ''}".strip(),
            f"{law_name} consolidated version latest amendment",
            f"{law_name} Rules Regulations",
        ]
    if hier.get("section"):
        seeds.append(f"{law_name or ''} section {hier['section']}".strip())
    if hier.get("subsection"):
        seeds.append(f"{law_name or ''} subsection {hier['subsection']}".strip())
    if hier.get("clause"):
        seeds.append(f"{law_name or ''} clause {hier['clause']}".strip())

    out, seen = [], set()
    for s in seeds:
        s = " ".join(str(s).split())
        if s and s.lower() not in seen:
            out.append(s); seen.add(s.lower())
    return out

def retrieve_from_index(deps: Deps, index_key: str, state: RAGState, top_k: int = 8) -> List[Hit]:
    index_name = deps.indexes[index_key]
    hits: List[Hit] = []
    for q in build_queries(state):
        vec = embed(deps, q)
        try:
            matches = pc_query(deps, index_name, vec, top_k=top_k)
        except Exception as e:
            print(f"[WARN] Pinecone query failed for {index_name}: {e}")
            continue
        for m in matches:
            md = m.get("metadata", {}) or {}
            txt = md.get("page_content") or md.get("text") or ""
            score = float(m.get("score", 0.0))
            hits.append({
                "index": index_key,
                "id": m.get("id", ""),
                "score": score,
                "text": txt,
                "metadata": {**md, "_index_name": index_name, "_query": q}
            })
    return hits

def mk_citation(h: Hit) -> Dict[str, Any]:
    md = h.get("metadata", {}) or {}
    title = md.get("title") or md.get("doc_title") or md.get("rules_name") or md.get("act_title") or ""
    link = md.get("source_url") or md.get("url") or md.get("source") or md.get("pdf_url") or ""
    location = md.get("section") or md.get("rule") or md.get("clause") or ""
    return {
        "index": h.get("index"),
        "id": h.get("id"),
        "title": title,
        "location": location,
        "score": h.get("score"),
        "link": link,
    }


In [6]:
def node_extract_entities(state: RAGState, deps: Deps) -> RAGState:
    chain = ENTITIES_PROMPT | deps.llm_fast | StrOutputParser()
    raw = chain.invoke({"query": state["query"]})
    try:
        data = json.loads(_strip_json(raw))
    except Exception:
        data = {"law_name": None, "law_year": None, "hierarchy": {}}
    hier = data.get("hierarchy") or {}
    for k in ["part", "chapter", "section", "subsection", "clause"]:
        hier[k] = hier.get(k)
    return {"entities": {"law_name": data.get("law_name"), "law_year": data.get("law_year"), "hierarchy": hier}}

def node_retrieve_acts(state: RAGState, deps: Deps) -> RAGState:
    return {"acts_hits": retrieve_from_index(deps, "acts", state, top_k=10)}

def node_synthesize_stage1(state: RAGState, deps: Deps) -> RAGState:
    acts_sorted = sorted(state.get("acts_hits", []), key=lambda h: h["score"], reverse=True)[:8]
    context = "\n\n---\n\n".join(h["text"][:1800] for h in acts_sorted)
    chain = SYNTH_STAGE1 | deps.llm_reason | StrOutputParser()
    answer = chain.invoke({"query": state["query"], "context": context}).strip()
    citations = [mk_citation(h) for h in acts_sorted[:4]]
    return {"stage1_answer": answer, "citations_stage1": citations}

def node_decide_escalation(state: RAGState, deps: Deps) -> RAGState:
    chain = ESCALATE_PROMPT | deps.llm_fast | StrOutputParser()
    raw = chain.invoke({"query": state["query"], "answer": state.get("stage1_answer","")}).strip()
    try:
        plan = json.loads(_strip_json(raw))
    except Exception:
        plan = {
            "need_rules": True, "need_sro": True, "need_ordinances": False,
            "need_web": True, "targeted_queries": []
        }
    return {"plan": plan}


In [16]:
def node_retrieve_rules(state: RAGState, deps: Deps) -> RAGState:
    plan = state.get("plan", {}) or {}
    if not plan.get("need_rules"):
        return {}
    return {"rules_hits": retrieve_from_index(deps, "rules", state, top_k=8)}

def node_retrieve_sro(state: RAGState, deps: Deps) -> RAGState:
    plan = state.get("plan", {}) or {}
    if not plan.get("need_sro"):
        return {}
    return {"sro_hits": retrieve_from_index(deps, "sro", state, top_k=8)}

def node_retrieve_ordinances(state: RAGState, deps: Deps) -> RAGState:
    plan = state.get("plan", {}) or {}
    if not plan.get("need_ordinances"):
        return {}
    return {"ordinances_hits": retrieve_from_index(deps, "ordinances", state, top_k=8)}

def node_web_search(state: RAGState, deps: Deps) -> RAGState:
    plan = state.get("plan", {}) or {}
    if not (state.get("use_web", True) and plan.get("need_web")):
        return {}
    api_key = os.environ.get("TAVILY_API_KEY")
    if not api_key:
        return {}
    ents = state.get("entities", {}) or {}
    wq = f"{ents.get('law_name') or ''} {ents.get('law_year') or ''} latest SRO site:nbr.gov.bd".strip() or state["query"]
    payload = {
        "api_key": api_key,
        "query": wq,
        "max_results": 6,
        "include_answer": False,
        "include_images": False,
        "include_domains": deps.allowed_domains,
        "search_depth": "advanced",
    }
    hits: List[Hit] = []
    try:
        resp = requests.post(deps.tavily_endpoint, json=payload, timeout=30)
        data = resp.json()
        for i, r in enumerate(data.get("results", [])):
            url = r.get("url", "")
            snippet = r.get("content", "") or r.get("snippet", "")
            title = r.get("title", "") or url
            hits.append({
                "index": "web",
                "id": url,
                "score": 0.5 + (6 - i) * 0.05,
                "text": snippet[:1800],
                "metadata": {"title": title, "source_url": url, "provider": "tavily"}
            })
    except Exception as e:
        print(f"[WARN] Tavily web search failed: {e}")
    return {"web_hits": hits}


In [8]:
def node_merge_hits(state: RAGState, deps: Deps) -> RAGState:
    merged: List[Hit] = []
    for key in ["acts_hits", "rules_hits", "sro_hits", "ordinances_hits", "web_hits"]:
        src = sorted(state.get(key, []) or [], key=lambda h: h["score"], reverse=True)[:6]
        merged.extend(src)
    seen = set(); uniq: List[Hit] = []
    for h in merged:
        k = (h["index"], h["id"])
        if k in seen: continue
        seen.add(k); uniq.append(h)
    return {"merged_hits": uniq[:18]}

def node_verify_latest(state: RAGState, deps: Deps) -> RAGState:
    ctx = "\n\n---\n\n".join([h["text"][:1600] for h in state.get("merged_hits", [])])
    chain = VERIFY_PROMPT | deps.llm_fast | StrOutputParser()
    raw = chain.invoke({"query": state["query"], "context": ctx}).strip()
    try:
        v = json.loads(_strip_json(raw))
    except Exception:
        v = {"notes": "Verification unavailable", "risk_level": "medium", "must_check": []}
    return {"verification": v}

def node_synthesize_final(state: RAGState, deps: Deps) -> RAGState:
    ctx = "\n\n---\n\n".join([h["text"][:1600] for h in state.get("merged_hits", [])])
    chain = FINAL_SYNTH | deps.llm_reason | StrOutputParser()
    ans = chain.invoke({
        "query": state["query"],
        "context": ctx,
        "verify": json.dumps(state.get("verification", {}), ensure_ascii=False)
    }).strip()
    cites: List[Dict[str, Any]] = []
    for h in (state.get("merged_hits") or [])[:10]:
        c = mk_citation(h)
        if c: cites.append(c)
    return {"final_answer": ans, "citations": cites}


In [17]:
def route_need_rules(state: RAGState) -> bool:
    return bool(state.get("plan", {}).get("need_rules"))

def route_need_sro(state: RAGState) -> bool:
    return bool(state.get("plan", {}).get("need_sro"))

def route_need_ordinances(state: RAGState) -> bool:
    return bool(state.get("plan", {}).get("need_ordinances"))

def route_need_web(state: RAGState) -> bool:
    return bool(state.get("plan", {}).get("need_web") and state.get("use_web", True))

def build_graph(indexes: Dict[str, str] = None,
                model_fast: str = "gpt-5-mini",
                model_reason: str = "gpt-5",
                embedding_model: str = "text-embedding-3-large",
                allowed_domains: List[str] = None):
    deps = Deps(
        indexes=indexes or DEFAULT_INDEXES,
        model_fast=model_fast,
        model_reason=model_reason,
        embedding_model=embedding_model,
        allowed_domains=allowed_domains or ALLOWED_DOMAINS_DEFAULT
    )
    deps.init_clients()

    g = StateGraph(RAGState)

    # Nodes
    g.add_node("extract_entities", lambda s: node_extract_entities(s, deps))
    g.add_node("retrieve_acts", lambda s: node_retrieve_acts(s, deps))
    g.add_node("synthesize_stage1", lambda s: node_synthesize_stage1(s, deps))
    g.add_node("decide_escalation", lambda s: node_decide_escalation(s, deps))

    # GATED sequential nodes (each checks plan and no-ops if not needed)
    g.add_node("retrieve_rules", lambda s: node_retrieve_rules(s, deps))
    g.add_node("retrieve_sro", lambda s: node_retrieve_sro(s, deps))
    g.add_node("retrieve_ordinances", lambda s: node_retrieve_ordinances(s, deps))
    g.add_node("web_search", lambda s: node_web_search(s, deps))

    g.add_node("merge_hits", lambda s: node_merge_hits(s, deps))
    g.add_node("verify_latest", lambda s: node_verify_latest(s, deps))
    g.add_node("synthesize_final", lambda s: node_synthesize_final(s, deps))

    # Edges (linear)
    g.add_edge(START, "extract_entities")
    g.add_edge("extract_entities", "retrieve_acts")
    g.add_edge("retrieve_acts", "synthesize_stage1")
    g.add_edge("synthesize_stage1", "decide_escalation")

    # Always run in sequence; nodes gate themselves
    g.add_edge("decide_escalation", "retrieve_rules")
    g.add_edge("retrieve_rules", "retrieve_sro")
    g.add_edge("retrieve_sro", "retrieve_ordinances")
    g.add_edge("retrieve_ordinances", "web_search")
    g.add_edge("web_search", "merge_hits")

    g.add_edge("merge_hits", "verify_latest")
    g.add_edge("verify_latest", "synthesize_final")
    g.add_edge("synthesize_final", END)

    # If you want to avoid thread_id requirement, compile without checkpointer:
    # app = g.compile()
    # Otherwise keep MemorySaver and pass thread_id in invoke:
    memory = MemorySaver()
    app = g.compile(checkpointer=memory)
    return app


In [19]:
def show_result(s: dict):
    """Safely print the final answer, citations, and verification from a LangGraph state dict."""
    print("\n--- FINAL ANSWER ---")
    print(s.get("final_answer", "[no final_answer key — the graph may have ended early]"))

    print("\n--- CITATIONS ---")
    cites = s.get("citations", [])
    if not cites:
        print("[no citations]")
    else:
        for i, c in enumerate(cites, 1):
            print(f"{i}. {c}")

    print("\n--- VERIFICATION ---")
    print(s.get("verification", "[no verification]"))


In [22]:
app = build_graph(indexes=DEFAULT_INDEXES)

state = app.invoke(
    {
        "query": "ধারা ১৬৫ অনুযায়ী withholding tax rate কত?",
        "use_web": True,
    },
    config={"configurable": {"thread_id": "alif-session-001"}}
)

show_result(state)  # (the helper you already defined)



--- FINAL ANSWER ---
সংক্ষিপ্ত উত্তর: ধারা ১৬৫ নিজে কোনো “একক” হার দেয় না। আইনে বলা আছে—উৎসে কর “বিধিমালায় নির্ধারিত হারে” কর্তন হবে। তাই হারটি সংশ্লিষ্ট পরিশোধ/সেবা/পণ্য–শ্রেণির উপর নির্ভর করে এবং বর্তমান হারগুলো উৎসে কর বিধিমালা, ২০২৪-এ (সর্বশেষ সংশোধন: ২৭ জুন ২০২৪) সারণী অনুযায়ী প্রযোজ্য। [Act §165], [Rules r.5/Table; SRO 248/2024]

প্রচলিত কিছু হার (উদাহরণ)
- উপদেষ্টা/পরামর্শক ফি: স্বাভাবিক ব্যক্তি 20%; অন্যান্য ব্যক্তি 10% [Rules r.5/Table; SRO 248/2024]
- পেশাদার সেবা: স্বাভাবিক ব্যক্তি 20%; অন্যান্য ব্যক্তি 10% [Rules r.5/Table; SRO 248/2024]
- প্রি-শিপমেন্ট পরিদর্শন: 20% [Rules r.5/Table; SRO 248/2024]
- কারিগরি সেবা/টেকনিক্যাল নো-হাউ/টেকনিক্যাল সহায়তা: 20% [Rules r.5/Table; SRO 248/2024]
- আইনি সেবা: 20% [Rules r.5/Table; SRO 248/2024]
- ইভেন্ট ম্যানেজমেন্ট/ম্যানেজমেন্ট সেবা: 20% [Rules r.5/Table; SRO 248/2024]
- কমিশন: 20% [Rules r.5/Table; SRO 248/2024]

পণ্য/সরবরাহ-সংক্রান্ত (নির্বাচিত) হার
- এমএস বিলেট উৎপাদন/স্থানীয় এমএস স্ক্র্যাপ ক্রয়: 0.5% [Rules r.3/Table; SRO 2

In [25]:
%pip install grandalf

  Using cached grandalf-0.8-py3-none-any.whl.metadata (1.7 kB)
Using cached grandalf-0.8-py3-none-any.whl (41 kB)

   ---------------------------------------- 0/2 [pyparsing]
   ---------------------------------------- 0/2 [pyparsing]
   -------------------- ------------------- 1/2 [grandalf]
   ---------------------------------------- 2/2 [grandalf]

Note: you may need to restart the kernel to use updated packages.


In [26]:
# assumes `app = build_graph(indexes=DEFAULT_INDEXES)` already ran
ascii_map = app.get_graph().draw_ascii()
print(ascii_map)


     +-----------+       
     | __start__ |       
     +-----------+       
            *            
            *            
            *            
  +------------------+   
  | extract_entities |   
  +------------------+   
            *            
            *            
            *            
   +---------------+     
   | retrieve_acts |     
   +---------------+     
            *            
            *            
            *            
 +-------------------+   
 | synthesize_stage1 |   
 +-------------------+   
            *            
            *            
            *            
 +-------------------+   
 | decide_escalation |   
 +-------------------+   
            *            
            *            
            *            
  +----------------+     
  | retrieve_rules |     
  +----------------+     
            *            
            *            
            *            
    +--------------+     
    | retrieve_sro |     
    +-------

In [27]:
state = app.invoke(
    {
        "query": "ইভেন্ট ম্যানেজমেন্ট সেবার কমিশন ফীর উপর টি ডি এস রেট কতো ?",
        "use_web": True,
    },
    config={"configurable": {"thread_id": "alif-session-001"}}
)

show_result(state)  # (the helper you already defined)



--- FINAL ANSWER ---
সংক্ষিপ্ত উত্তর: ইভেন্ট ম্যানেজমেন্ট সেবার “কমিশন/ফি” এর উপর টি.ডি.এস. 10%। যদি কমিশন/ফি আলাদা না দেখিয়ে মোট বিলের উপর কাটা হয়, হার 2%। বিদেশি (নন-রেসিডেন্ট) সেবা-দাতাকে এ ধরনের ব্যবস্থাপনা সেবার ফি দিলে সাধারণত 20% কাটা হয় (প্রযোজ্য ডিটিএএ থাকলে তা অনুযায়ী সামঞ্জস্য হবে)।

আইনগত ভিত্তি
- Act: নির্দিষ্ট ব্যক্তি কর্তৃক নিবাসীকে সেবা বাবদ পরিশোধে উৎসে কর কর্তনের ক্ষমতা [Act §90]
- Rules:
  - ইভেন্ট পরিচালনা/ম্যানেজমেন্টসহ সেবা—(১) কমিশন/ফি এর উপর 10%, (২) মোট বিলের উপর 2% [Rule 4(1) Table, item 4(1)-(2)]
  - নন-রেসিডেন্টকে ব্যবস্থাপনা/ইভেন্ট ম্যানেজমেন্ট সেবা বাবদ পরিশোধে 20% [Rule 5(1) Table, item 9]

ব্যবহারিক টিপস
- ক্রেডিট বা পেমেন্ট—যেটি আগে ঘটে—সে সময় কেটে জমা দিন।
- কমিশন/ফি আলাদা লাইনে থাকলে 10%; না থাকলে মোট বিলভিত্তিক 2% নিন—ডকুমেন্টেশনে স্পষ্টতা রাখুন।
- নন-রেসিডেন্ট হলে ডিটিএএ থাকলে হার/স্কোপ ভিন্ন হতে পারে—প্রযোজ্যতা যাচাই করুন।

--- CITATIONS ---
1. {'index': 'acts', 'id': '7ed407f0-97bd-46e3-9661-6ecfbcb4db2a', 'title': '', 'location': '', 'score': 0.

In [28]:
state = app.invoke(
    {
        "query": "ভাড়ার উপর টি ডি এস রেট কতো?",
        "use_web": True,
    },
    config={"configurable": {"thread_id": "alif-session-001"}}
)

show_result(state)  # (the helper you already defined)


--- FINAL ANSWER ---
সংক্ষিপ্ত উত্তর (সাধারণ বাড়ি/অফিস/দোকান/ফ্লোর স্পেস/গুদাম ইত্যাদি ভাড়া): ৫% উৎসে কর (TDS)।

ব্যাখ্যা ও প্রয়োগ
- হার: অস্থাবর সম্পত্তির (immovable property) ভাড়ার উপর সাধারণত ৫% হারে উৎসে কর কাটা হয়। [Act §89], [Withholding Tax Rules 2024, SRO 248/2024]
- কখন কাটবেন: ভাড়া হিসাবভুক্ত (credit) বা পরিশোধ—যেটি আগে ঘটে। [Act §89]
- ভিত্তিমূল্য: ভাড়ার অঙ্ক (সাধারণত ভ্যাট ব্যতীত); অগ্রিম ভাড়া/ডিপোজিট যেটি ভাড়ার অংশ হিসেবে প্রদান করা হয়, তার ওপরও প্রযোজ্য। [WTR 2024, SRO 248/2024]
- কারা কাটবেন: সরকার/স্বায়ত্তশাসিত সংস্থা/কোম্পানি/ফার্ম/এওপি/এনজিও/সহযোগী সমিতি প্রভৃতি নির্ধারিত কর্তনকারী সত্তা। ব্যক্তিগতভাবে বাসা ভাড়া নেওয়া সাধারণ ব্যক্তির ক্ষেত্রে বাধ্যতামূলক নয়। [Act §89], [WTR 2024]
- ব্যতিক্রম/ছাড়: সরকারকে প্রদেয় বা আইনগতভাবে অব্যাহতি-প্রাপ্ত আয়ের ক্ষেত্রে (ধারা ৭৬(৫)-(৬) অনুযায়ী প্রমাণপত্র থাকলে) TDS নাও লাগতে পারে। [Act §76], [SRO 340/2024]
- নন-ফাইলার (রিটার্ন দাখিল না করলে): আইনে বর্ধিত হারে কর্তনের বিধান আছে; প্রযোজ্য হলে নির্ধারিত হারের চেয়ে বেশি কেটে নিতে হয়—এ ব

In [29]:
state = app.invoke(
    {
        "query": "বনানীতে গ-শ্রেনীর ভূমির উপর করহার কত?",
        "use_web": True,
    },
    config={"configurable": {"thread_id": "alif-session-001"}}
)

show_result(state)  # (the helper you already defined)


--- FINAL ANSWER ---
সোজা উত্তর
- বনানী এলাকার গ-শ্রেণির (ডেভেলপার/রিয়েল এস্টেট ডেভেলপার কর্তৃক প্রতিষ্ঠিত এলাকা—বাণিজ্যিক প্লট) ভূমির উপর করহার: দলিলমূল্যের 5% অথবা প্রতি শতকে টাকা 9,00,000—যেটি বেশি [Rules: Mouza-wise Table-1; sub-rule (8)(গ)]।

গুরুত্বপূর্ণ ব্যাখ্যা
- গ-শ্রেণি মানে: RAJUK/CDA/GDA/NHA/গণপূর্ত/ক্যান্টনমেন্ট বোর্ডের নিয়ন্ত্রণাধীন নয় এমন এলাকায় ডেভেলপার দ্বারা প্রতিষ্ঠিত বাণিজ্যিক প্লট [Rules sub-rule (8)(গ)]।
- বনানী সাধারণত RAJUK নিয়ন্ত্রিত। সে ক্ষেত্রে শ্রেণি সাধারণত ক (বাণিজ্যিক) বা খ (আবাসিক) হবে, যার হার টেবিলে ভিন্ন। আপনি প্রকৃত শ্রেণি (ক/খ/গ) নিশ্চিত করুন, নচেৎ হার বদলে যাবে [Rules: Table-1]।

নোট (সমসাময়িক আপডেট সম্পর্কে)
- একাধিক সংকলনে একই টেবিলের একটি সংস্করণে উপরের এলাকাগুলোর জন্য 6% দেখা যায়; অন্যটিতে 5%। সর্বশেষ গেজেট/প্রজ্ঞাপনের ভিত্তিতে হার নির্ধারণ করা উচিত। আপনার দলিলের তারিখ দিলে প্রযোজ্য কার্যকর-তারিখ অনুসারে হার মিলিয়ে দিতে পারবো [Act (enabling), Rules Table-1; recent SROs 2024 noted]. 

সারসংক্ষেপে, আপনি যদি সত্যিই “গ-শ্রেণি (Banani)” প্লট ট্রান্

In [30]:
state = app.invoke(
    {
        "query": "শরীয়তপুর জেলার নড়িয়া উপজেলার ভূমির উপর করহার কত?",
        "use_web": True,
    },
    config={"configurable": {"thread_id": "alif-session-001"}}
)

show_result(state)  # (the helper you already defined)


--- FINAL ANSWER ---
সংক্ষেপে (উৎসে কর/রেজিস্ট্রেশনকালে)
- নড়িয়া উপজেলার গ্রাম/ইউনিয়ন (পৌরসভা-এলাকা নয়): দলিলে উল্লিখিত ভূমিমূল্যের 2% [Act §125], [Rule 6, Table‑2(3)]।
- নড়িয়া পৌরসভা এলাকার মধ্যে হলে: দলিলে উল্লিখিত ভূমিমূল্যের 4% [Act §125], [Rule 6, Table‑2(2)]।
- যদি দলিলের ভূমির উপর স্থাপনা/বাড়ি/ফ্ল্যাট থাকে: উপরের হারের সঙ্গে অতিরিক্ত কর প্রযোজ্য—“অন্যান্য এলাকা” (শ্রেণি-ঙ) ক্ষেত্রে সাধারণত প্রতি বর্গমিটারে 500 টাকা অথবা দলিলে উল্লিখিত স্থাপনার মূল্যের 6%—যেটি বেশি, সেটি [Rule 6(2), Structure Table item 2]।

ব্যবহারিক টীকা
- রেজিস্ট্রেশন কর্মকর্তা পে-অর্ডারের মাধ্যমে কর পরিশোধের প্রমাণ ছাড়া দলিল নিবন্ধন করবেন না [Rule 6(1)]।
- ডেভেলপার/রিয়েল এস্টেট ডেভেলপারের কাছ থেকে ফ্ল্যাট/অ্যাপার্টমেন্ট কিনলে ধারা 126 অনুযায়ী আলাদা প্রতি-বর্গমিটার হার প্রযোজ্য; এলাকার ওপর নির্ভর করে হার নির্ধারিত থাকে [Act §126], [Rule 7, Schedule]।
- উপরোক্ত হারগুলো আয়কর আইন, ২০২৩-এর অধীনে সম্পত্তি হস্তান্তরজনিত উৎসে কর। আপনি যদি “ভূমি উন্নয়ন কর” (বার্ষিক LDT) বোঝাতে চান, সেটি পৃথক আইন/হার—জানালে তা আলাদা 

In [ ]:
state = app.invoke(
    {
        "query": "পটেটো চিপস এর উপর সম্পূরক শুল্কহার কতো? HS Code 2005.20.00",
        "use_web": False,
    },
    config={"configurable": {"thread_id": "alif-session-001"}}
)

show_result(state)  # (the helper you already defined)


--- FINAL ANSWER ---
সংক্ষিপ্ত উত্তর: HS Code 2005.20.00 (Potato chips)–এর উপর বর্তমান সম্পূরক শুল্ক (SD) হার 0% (অর্থাৎ SD প্রযোজ্য নয়)। এটি আমদানি ও স্থানীয় সরবরাহ—উভয় ক্ষেত্রেই একই।

আইনি ভিত্তি ও সূত্র:
- VAT & SD Act, 2012—আইনের অধীন সম্পূরক শুল্ক কেবল First Schedule-এ তালিকাভুক্ত পণ্যের উপর আরোপিত হয়; HS 2005.20.00 বর্তমানে ওই তালিকায় SD-যুক্ত নয় [VAT & SD Act 2012, First Schedule].
- বার্ষিক বাজেট/Finance Act অনুযায়ী First Schedule হালনাগাদ হয়; 2024-25 অর্থবছরের হালনাগাদে উক্ত HS-এ SD সংযোজন নেই [VAT & SD Act 2012, First Schedule (as amended FY 2024-25)].
- সর্বশেষ কার্যকর সময়কার হার যাচাইয়ের জন্য NBR-এর ট্যারিফ/গেজেট দেখুন (Customs–VAT integrated tariff table) [NBR: nbr.gov.bd].

ব্যবহারিক নোট:
- SD 0% হলেও স্ট্যান্ডার্ড VAT সাধারণত প্রযোজ্য থাকে; আমদানিতে অন্যান্য শুল্ক/কর (CD, RD, AIT ইত্যাদি) আলাদা বিষয়।
- মধ্যবর্তী সময়ে কোনো নতুন SRO/গেজেট জারি হলে হার পরিবর্তিত হতে পারে; বিল অব এন্ট্রি করার আগে NBR ট্যারিফের সংশ্লিষ্ট সারি দেখে নিন বা কাস্টমস EDI/LC পরামর্শদাতার মাধ্যমে